# Using NLP AI model to refine classification
> Base on the keyword rule based label

* I intend to build a neat dataset about ancient Chinese literatures, all the poem, article, history, etc.
* [Wikisource dump on zhongwen](https://dumps.wikimedia.org/zhwikisource/20200301/) is a wonderfule starting point
* The problem is, it contains too much morden text, include government decree, notice, law etc.

### So the task here is to classify the text

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/zhwikisource-title-draft/cntext_rule_sep.csv


The data was created from [this notebook](https://github.com/raynardj/python4ml/blob/master/experiments/ancient_cn/07_filter_out_morden_cn_part2.ipynb)

In [2]:
from pathlib import Path
DATA =  Path("/kaggle/input/zhwikisource-title-draft/cntext_rule_sep.csv")

In [3]:
df = pd.read_csv(DATA)

### Samples on positive label

In [4]:
df.sample(10)

,id,url,title,preview,isy
79773,843467,https://zh.wikisource.org/wiki?curid=843467,尚書正義 (四部叢刊本)/序,"尚書正義 (四部叢刊本)/序\n\n<pages from=""4"" to=""11"" inde...",True
7702,35269,https://zh.wikisource.org/wiki?curid=35269,喜聞太原同院崔侍御臺拜兼寄在臺三二同年之什,喜聞太原同院崔侍御臺拜兼寄在臺三二同年之什\n\n<onlyinclude>\n<poem>...,True
184361,211693,https://zh.wikisource.org/wiki?curid=211693,春秋大事表 (四庫全書本)/卷12上,春秋大事表 (四庫全書本)/卷12上\n\n<onlyinclude><poem> 欽定四...,True
294969,62488,https://zh.wikisource.org/wiki?curid=62488,次韻柳子玉/地爐,次韻柳子玉/地爐\n\n次韻柳子玉\n<onlyinclude>\n<poem>\n細聲蚯蚓...,True
87691,179408,https://zh.wikisource.org/wiki?curid=179408,前漢演義/第085回,前漢演義/第085回\n\n 卻說張敞久守山陽，境內無事，自覺閒暇得很。會聞渤海膠東，人民...,True
67566,108618,https://zh.wikisource.org/wiki?curid=108618,平山冷燕,平山冷燕\n\n,True
202637,257571,https://zh.wikisource.org/wiki?curid=257571,太𤣥本㫖 (四庫全書本)/卷8,太𤣥本㫖 (四庫全書本)/卷8\n\n<onlyinclude><poem> 欽定四庫全書...,True
119870,53595,https://zh.wikisource.org/wiki?curid=53595,同于汝錫賞白牡丹,同于汝錫賞白牡丹\n\n<onlyinclude>\n<poem>\n曉日花初吐，春寒白未凝...,True
157350,133453,https://zh.wikisource.org/wiki?curid=133453,賽雨紀石文,賽雨紀石文\n\n<onlyinclude>昭明為神，神正直而已。其或歆夫諂祭，阨我生人，則...,True
96223,158446,https://zh.wikisource.org/wiki?curid=158446,文昌雜錄/卷二,文昌雜錄/卷二\n\n十月，以天章閤待制王克臣為工部侍郎，熊本知桂州故也。以水部郎中李文卿為...,True


In [5]:
df = df.sample(frac = 1.).reset_index().drop("index",axis=1)

### Samples on negtive lables

In [6]:
df[~df.isy].sample(10)

,id,url,title,preview,isy
56672,187250,https://zh.wikisource.org/wiki?curid=187250,中华人民共和国国务院公报/2001年/第33号,中华人民共和国国务院公报/2001年/第33号\n,False
269815,142618,https://zh.wikisource.org/wiki?curid=142618,全国人民代表大会关于香港特别行政区第一届政府和立法会产生办法的决定,全国人民代表大会关于香港特别行政区第一届政府和立法会产生办法的决定\n\n 一、香港特别行...,False
138128,176369,https://zh.wikisource.org/wiki?curid=176369,国家公路网规划（2013年－2030年）,国家公路网规划（2013年－2030年）\n\n《中华人民共和国公路法》（第一章、第六条）明...,False
126790,884524,https://zh.wikisource.org/wiki?curid=884524,企业会计准则第20号——企业合并 (2006年),企业会计准则第20号——企业合并 (2006年)\n\n<onlyinclude>\n\n ...,False
124691,21664,https://zh.wikisource.org/wiki?curid=21664,大理院統字第443號解釋,大理院統字第443號解釋\n\n 共同實施犯罪行為應以共同正犯論唯行為有輕重之別則處刑亦自...,False
278352,23980,https://zh.wikisource.org/wiki?curid=23980,大理院統字第1069號解釋,大理院統字第1069號解釋\n\n 田畝未經報戶入冊應照國有荒地承墾條例辦理,False
71429,93034,https://zh.wikisource.org/wiki?curid=93034,行政院組織法 (民國37年),行政院組織法 (民國37年)\n\n第一條\n第二條\n第三條\n第四條\n第五條\n第六條...,False
185443,59788,https://zh.wikisource.org/wiki?curid=59788,司法院院解字第3474號解釋,司法院院解字第3474號解釋\n\n 甲告訴乙犯偽造文書等罪，經檢察官認有誣告嫌疑將其羈押...,False
299695,1005872,https://zh.wikisource.org/wiki?curid=1005872,行政訴訟法施行法 (民國108年立法109年公布),行政訴訟法施行法 (民國108年立法109年公布)\n\n第一條 (新、舊法之定義)\n第二...,False
82837,48769,https://zh.wikisource.org/wiki?curid=48769,司法院院字第1221號解釋,司法院院字第1221號解釋\n\n 原呈所稱判處罰金刑之案件，為被告利益起見而提起非常上訴...,False


In [7]:
from transformers import BertTokenizer,BertModel

In [8]:
tok = BertTokenizer.from_pretrained("bert-base-chinese")

In [9]:
tok.encode_plus("最高人民法院关于处理自首和立功具体应用法律若干问题的解释"),
tok.encode_plus("欽定八旗通志 (四庫全書本)/卷021")

{'input_ids': [101,
  3620,
  2137,
  1061,
  3186,
  6858,
  2562,
  113,
  1724,
  2430,
  1059,
  3292,
  3315,
  114,
  120,
  1318,
  10385,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
bert = BertModel.from_pretrained("bert-base-chinese")

In [11]:
import torch

In [12]:
test_toks = tok.encode_plus("欽定八旗通志 (四庫全書本)/卷021")["input_ids"]
test_x = torch.LongTensor(test_toks)[None,:]

In [13]:
with torch.no_grad():
    y_1,y_2 = bert(test_x)

In [14]:
y_1.shape

torch.Size([1, 18, 768])

In [15]:
y_2.shape

torch.Size([1, 768])

In [16]:
CUDA = torch.cuda.is_available()

In [17]:
if CUDA:
    bert.cuda()

In [18]:
from torch import nn
class newTop(nn.Module):
    def __init__(self,in_ = 768,hs = 768):
        super().__init__()
        self.ff = nn.Sequential(*[
            nn.Linear(in_,hs,bias = False),
            nn.BatchNorm1d(hs),
            nn.ReLU(),
            nn.Linear(hs,1)
        ])
        
    def forward(self,x):
        return self.ff(x)

### Pytorch dataset

> a pytorch dataset, generating tokenized and encoded textual info and y label

In [19]:
from torch.utils.data.dataset import Dataset

class TextDS(Dataset):
    def __init__(self,df):
        self.df = df
        self.pre = list(df.preview)
        self.y = list(df.isy)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        txt = self.pre[idx]
        ids = tok.encode(txt,max_length=100,pad_to_max_length=True)
        return np.array(ids),np.array([1 if self.y[idx] else 0,])

### Forgebox train loop toolkit
This is my personal toolkit, feel welcomed to use other solutions

In [21]:
!pip install forgebox

In [22]:
from forgebox.ftorch.train import Trainer
from forgebox.ftorch.callbacks import stat

In [55]:
t = Trainer(TextDS(df),val_dataset=TextDS(df),shuffle=False,
        batch_size = 32, print_on = 5, 
        callbacks=[stat],val_callbacks=[stat],using_gpu = CUDA)

Notice, The Trainer was not initiated with optimizer
            Use the following syntax to initialize optimizer
            t.opt["adm1"] = torch.optim.Adam(m1.parameters())
            t.opt["adg1"] = torch.optim.Adagrad(m2.parameters())
            


In [24]:
model = newTop()
if CUDA:
    model.cuda()

We only train the top layer module

In [56]:
t.opt["adm_top"] = torch.optim.Adam(model.parameters())

In [65]:
# The loos function that will reduce down the loss to a single scalar/ mini-batch
lossf = nn.BCEWithLogitsLoss()
# The loss function that will keep the dimension on mini-batch
loss_e = nn.BCEWithLogitsLoss(reduce = False)

In [59]:
from forgebox.ftorch.metrics import metric4_bi

Recording every loss element on every line

In [60]:
class Looper:
    def __init__(self):
        self.record = []
        
    def __call__(self,*args):
        for arg in args:
            self.record.append(arg)

In [61]:
@t.step_train
def action(batch):
    if batch.i == 0: batch.l = Looper()
    batch.opt.zero_all()
    x,y = batch.data
    x = x.long()
    y = y.float()

    with torch.no_grad():
        _,vec = bert(x)
    y_ = model(vec)
    loss = lossf(y_,y)
    
    loss.backward()
    batch.opt.step_all()
    acc,rec,prec,f1 = metric4_bi(y_,y)
    return {"loss":loss.item(),
            "f1":f1.item(),
            "acc":acc.item(),
            "rec":rec.item(),
            "prec":prec.item()}

@t.step_val
def val_action(batch):
    if batch.i == 0: 
        batch.l = Looper()
        bert.eval()
        model.eval()
    with torch.no_grad():
        x,y = batch.data
        x = x.long()
        y = y.float()

        _,vec = bert(x)
        y_ = model(vec.detach())
        loss = loss_e(y_,y)
    
    batch.l(*loss.cpu().numpy())
    return {"loss":loss.mean().item(),}

### Train and eval
* Train the model according to the preliminary label
* Save the loss to the list 

In [ ]:
t.train(1)

### Save top layer model

In [ ]:
torch.save(model.state_dict(),"top_layer.pth")

In [63]:
t.epoch = 0
t.val_track[0] = list()
t.val_gen = iter(t.val_data)

val_t = t.progress(t.val_len)
for i in val_t:
    t.val_iteration(i, val_t)

for v_cb_func in t.val_callbacks:
    v_cb_func(record=t.val_track[0] )

,loss,epoch,iter,timestamp
mean,0.028367,0.0,4870.5,0.189960
min,0.000034,0.0,0.0,0.000000
max,0.579924,0.0,9741.0,1850.593469


Attach loss to dataframe

In [67]:
df["loss"] = t.l.record
df["loss"] = df.loss.apply(lambda x:x[0])

### Review the toploss rows

In [68]:
df.sort_values(by = "loss",ascending = False)

,id,url,title,preview,isy,loss
215525,155984,https://zh.wikisource.org/wiki?curid=155984,新約讀範,新約讀範\n\n西歷一千九百十二年\n\n中華民國元年壬子夏 上海廣學會出版\n\n上海商務...,False,15.630446
198650,27272,https://zh.wikisource.org/wiki?curid=27272,革命軍序,革命軍序\n\n蜀為《革命軍》，方二萬言，示余曰：「欲以立懦夫，定民志，故辭多恣肆，無所回避...,False,15.038565
56271,39775,https://zh.wikisource.org/wiki?curid=39775,新聞記者法,新聞記者法\n,False,14.737231
13389,821252,https://zh.wikisource.org/wiki?curid=821252,清史紀事本末 (民國三年石印本),清史紀事本末 (民國三年石印本)\n\n,False,13.385850
195876,885446,https://zh.wikisource.org/wiki?curid=885446,皇朝故潞州司法秦君墓志銘並序,皇朝故潞州司法秦君墓志銘並序\n\n<onlyinclude>\n公諱佾，齊國臨淄人也。高陽...,False,13.173684
...,...,...,...,...,...,...
208531,306655,https://zh.wikisource.org/wiki?curid=306655,夷白齋稿 (四庫全書本)/卷09,夷白齋稿 (四庫全書本)/卷09\n\n<onlyinclude><poem> 欽定四庫全...,True,0.000000
241187,267282,https://zh.wikisource.org/wiki?curid=267282,稗編 (四庫全書本)/卷037,稗編 (四庫全書本)/卷037\n\n<onlyinclude><poem> 欽定四庫全書...,True,0.000000
26401,254823,https://zh.wikisource.org/wiki?curid=254823,大統厯志 (四庫全書本)/卷8,大統厯志 (四庫全書本)/卷8\n\n<onlyinclude><poem> 欽定四庫全書...,True,0.000000
12722,76287,https://zh.wikisource.org/wiki?curid=76287,詠雨,詠雨\n,True,0.000000


### Save the loss to csv

In [69]:
df.sort_values(by = "loss",ascending = False).reset_index().drop("index",axis=1).to_csv("cntext_loss.csv",index = False)